# Download data in the region of interest using `Beacon`

__Notes:__
1. When getting data from Argo, we need to be sure that we have depth instead of pressure!7
2. We get more data when using `Beacon` than when using `pyargo`. Why is that?

In [1]:
using Pkg
Pkg.activate("../")
using HTTP
using JSON
using JSON3
using Dates
# using Makie, CairoMakie, GeoMakie
using NCDatasets
using OrderedCollections
include("./DIVAndFairEase.jl")
token = ENV["beaconAPItoken"];

  Activating project at `~/Projects/DIVAnd-FAIR-EASE`


In [2]:
figdir = "../figures/NorthAdriatic"
datadir = "../data/"
footprintdir = "../Footprint"
mkpath.([datadir, figdir, footprintdir])

3-element Vector{String}:
 "../data"
 "../figures/NorthAdriatic"
 "../Footprint"

## Beacon services

In [3]:
beacon_services = OrderedDict(
    "Euro-Argo" => "https://beacon-argo.maris.nl",
    "CORA Profile" => "https://beacon-cora-pr.maris.nl",
    "CORA Timeseries" => "https://beacon-cora-ts.maris.nl",
    "EMODnet Chemistry" => "https://beacon-emod-chem.maris.nl",
    "World Ocean Database" => "https://beacon-wod.maris.nl",
    "SeaDataNet CDI TS" => "https://beacon-cdi-ts.maris.nl",
    "CMEMS BGC" => "https://beacon-cmems.maris.nl",
)

OrderedDict{String, String} with 7 entries:
  "Euro-Argo"            => "https://beacon-argo.maris.nl"
  "CORA Profile"         => "https://beacon-cora-pr.maris.nl"
  "CORA Timeseries"      => "https://beacon-cora-ts.maris.nl"
  "EMODnet Chemistry"    => "https://beacon-emod-chem.maris.nl"
  "World Ocean Database" => "https://beacon-wod.maris.nl"
  "SeaDataNet CDI TS"    => "https://beacon-cdi-ts.maris.nl"
  "CMEMS BGC"            => "https://beacon-cmems.maris.nl"

In [4]:
datasourcelist = ["Euro-Argo", "World Ocean Database", "SeaDataNet CDI TS", "CORA Profile", "CORA Timeseries"]
@info("Working on $(length(datasourcelist)) data sources")
domain = [12., 18., 43., 46.]
datestart = Dates.Date(1950, 1, 1)
dateend = Dates.Date(2024, 1, 1)
minlon = domain[1]
maxlon = domain[2]
minlat = domain[3]
maxlat = domain[4]
mindepth = 0.
maxdepth = 500.
regionname = "NorthAdriaticSea"

[ Info: Working on 5 data sources


"NorthAdriaticSea"

In [5]:
datasourcelist

5-element Vector{String}:
 "Euro-Argo"
 "World Ocean Database"
 "SeaDataNet CDI TS"
 "CORA Profile"
 "CORA Timeseries"

## Loop on the data sources

In [6]:
include("./DIVAndFairEase.jl")
#for datasource in datasourcelist[3:3]
datasource = datasourcelist[3]    
@info("Working on $(datasource)")

parameter1 = "TEMP"
if datasource == "World Ocean Database"
    parameter1 = "Temperature"
elseif datasource == "EMODnet Chemistry"  
    parameter1 = "ITS_90_water_temperature"
elseif datasource == "SeaDataNet CDI TS"
    parameter1 = "ITS-90 water temperature"
end
parameter2 = "PSAL"

@info("$(datestart) → $(dateend), $(parameter1)")

# Download the "footprint"
beaconURL = beacon_services[datasource]
datasource_name = replace(datasource, " " => "_")

# Build query
query = DIVAndFairEase.prepare_query(datasource, parameter1, parameter2, Dates.Date(datestart), 
    Dates.Date(dateend), mindepth, maxdepth, minlon, maxlon, minlat, maxlat)

# Construct file and figure names
filename = joinpath(datadir, "$(regionname)_$(datasource_name)_$(replace(parameter1, " "=>"_"))_$(Dates.format(datestart, "yyyymmdd"))-$(Dates.format(dateend, "yyyymmdd"))_$(Int(mindepth))-$(Int(maxdepth))m.nc");
figname = "$(regionname)_$(datasource_name)_$(replace(parameter1, " "=>"_"))_$(Dates.format(datestart, "yyyymmdd"))-$(Dates.format(dateend, "yyyymmdd"))_$(Int(mindepth))-$(Int(maxdepth))m.jpg"

# Write the data in netCDF files
@info("Data will be written in file:\n$(filename)")
if isfile(filename)
    @info("File already downloaded")
    rm(filename)
end

@show(query)
@time open(filename, "w") do io
    r = HTTP.request("POST", joinpath(beaconURL, "api/query"), 
        ["Content-type"=> "application/json",
         "Authorization" => "Bearer $(token)"
        ],
        query, 
        response_stream=io);
    @info(r.status)
end

@info("NetCDF file size: $(round(filesize(filename)/1000^2, digits=1))M")

    # Create plot
# end   

[ Info: Working on SeaDataNet CDI TS
[ Info: 1950-01-01 → 2024-01-01, ITS-90 water temperature


query = "{\"query_parameters\":[{\"column_name\":\"ITS-90 water temperature\",\"alias\":\"ITS-90 water temperature\"},{\"column_name\":\"yyyy-mm-ddThh:mm:ss.sss\",\"alias\":\"TIME\"},{\"column_name\":\"Depth\",\"alias\":\"DEPTH\"},{\"column_name\":\"Longitude\",\"alias\":\"LONGITUDE\"},{\"column_name\":\"Latitude\",\"alias\":\"LATITUDE\"}],\"filters\":[{\"for_query_parameter\":\"TIME\",\"min\":\"19500101T00:00:00\",\"max\":\"20240101T00:00:00\"},{\"for_query_parameter\":\"DEPTH\",\"min\":0.0,\"max\":500.0},{\"for_query_parameter\":\"LONGITUDE\",\"min\":12.0,\"max\":18.0},{\"for_query_parameter\":\"LATITUDE\",\"min\":43.0,\"max\":46.0}],\"output\":{\"format\":\"netcdf\"}}"


┌ Info: Data will be written in file:
└ ../data/NorthAdriaticSea_SeaDataNet_CDI_TS_ITS-90_water_temperature_19500101-20240101_0-500m.nc
[ Info: File already downloaded


LoadError: HTTP.Exceptions.StatusError(400, "POST", "/api/query", HTTP.Messages.Response:
"""
HTTP/1.1 400 Bad Request
Content-Type: application/json
Vary: Access-Control-Request-Headers
x-beacon-query-id: 97130d37-6172-4a0c-b513-1a21368a8cae
X-Powered-By: ARR/3.0
Date: Wed, 11 Dec 2024 08:20:07 GMT
Content-Length: 76
Set-Cookie: ******

[Message Body was streamed]""")